In [21]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_columns', 100)
pd.set_option('max_rows', 200)
%matplotlib inline

# 训练集

In [22]:
train = pd.read_csv('train_pre.csv')

In [23]:
# train['gender'] = train['gender'].astype(int)
# train['age'] = train['age'].astype(int)

In [24]:
train_id = train[['user_id']]
train_y = train[['current_service']]
train_x = train.drop(['user_id', 'current_service'], axis=1)
train_feature = train_x.columns

In [25]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_y_le = le.fit_transform(train_y['current_service'])

In [26]:
x_train = train_x[1:int(4/5*len(train_x))]
x_valid = train_x[int(4/5*len(train_x)):]

In [31]:
y_train = train_y_le[1:int(4/5*len(train_x))]
y_valid = train_y_le[int(4/5*len(train_x)):]

In [32]:
train_data = lgb.Dataset(x_train, label=y_train)
valid_data = lgb.Dataset(x_valid, label=y_valid)

In [29]:
test = pd.read_csv('test_pre.csv')

In [30]:
test_id = test[['user_id']]
test_x = test.drop(['user_id'], axis=1)

## LGB程序

In [14]:
from sklearn.metrics import f1_score
import numpy as np

# 自定义F1评价函数
def f1_score_vali(preds, data_vali):
    labels = data_vali.get_label()
    preds = np.argmax(preds.reshape(11, -1), axis=0)
    score_vali = f1_score(y_true=labels, y_pred=preds, average='macro')
    return 'f1_score', score_vali**2, True

In [15]:
params = {'learning_rate':0.1, 
          'lambda_l1':0.18,
          'lambda_l2':0.23,
#           'max_depth':7,
#           'num_leaves':78, 
          'metric':None, 
          'objective':'multiclass',
          'num_class':11, 
          'silent':True
          }

In [33]:
best_params = {}
min_merror = float('Inf')
for max_depth in range(6, 9):
    for num_leaves in range(40, 110, 10):
        params['num_leaves'] = num_leaves
        params['max_depth'] = max_depth

        cv_results = lgb.cv(params, train_data, num_boost_round=100000, nfold=3, metrics=None,feval=f1_score_vali, early_stopping_rounds=50, verbose_eval=50)

        mean_merror = pd.Series(cv_results['multi_logloss-mean']).min()
        boost_rounds = pd.Series(cv_results['multi_logloss-mean']).argmin()

        if mean_merror < min_merror:
            min_merror = mean_merror
            best_params['num_leaves'] = num_leaves
            best_params['max_depth'] = max_depth

params['num_leaves'] = best_params['num_leaves']
params['max_depth'] = best_params['max_depth']
#             clf = lgb.train(params, train_data, num_boost_round=100000, valid_sets=[valid_data], feval=f1_score_vali, early_stopping_rounds=50, verbose_eval=50)
#             x_pred = clf.predict(x_valid, num_iteration=clf.best_iteration)
#             x_pred = [list(x).index(max(x)) for x in x_pred]
#             y_test = clf.predict(test_x, num_iteration=clf.best_iteration)
#             y_test = [list(x).index(max(x)) for x in y_test]

#     lgb_oof_train[valid_index] = x_pred
    
#     if index == 0:
#         cv_pred = np.array(y_test).reshape(-1, 1)
#     else:
#         cv_pred = np.hstack((cv_pred, np.array(y_test).reshape(-1, 1)))

[50]	cv_agg's multi_logloss: 0.281884 + 0.000801247	cv_agg's f1_score: 0.734143 + 0.00171883
[100]	cv_agg's multi_logloss: 0.224521 + 0.00111893	cv_agg's f1_score: 0.764764 + 0.00200017
[150]	cv_agg's multi_logloss: 0.206386 + 0.00132288	cv_agg's f1_score: 0.778558 + 0.00145749
[200]	cv_agg's multi_logloss: 0.197188 + 0.00127358	cv_agg's f1_score: 0.786015 + 0.00113123
[250]	cv_agg's multi_logloss: 0.191158 + 0.00116165	cv_agg's f1_score: 0.791599 + 0.00097261
[300]	cv_agg's multi_logloss: 0.186955 + 0.00105972	cv_agg's f1_score: 0.795788 + 0.00110792
[350]	cv_agg's multi_logloss: 0.183836 + 0.00100179	cv_agg's f1_score: 0.798844 + 0.000960048
[400]	cv_agg's multi_logloss: 0.181498 + 0.000962128	cv_agg's f1_score: 0.801334 + 0.000884025
[450]	cv_agg's multi_logloss: 0.17963 + 0.000918154	cv_agg's f1_score: 0.802924 + 0.00121136
[500]	cv_agg's multi_logloss: 0.178092 + 0.000923834	cv_agg's f1_score: 0.804937 + 0.000953138
[550]	cv_agg's multi_logloss: 0.176783 + 0.00093063	cv_agg's f1_s

[500]	cv_agg's multi_logloss: 0.176224 + 0.00099865	cv_agg's f1_score: 0.807637 + 0.00134771
[550]	cv_agg's multi_logloss: 0.174907 + 0.00097153	cv_agg's f1_score: 0.809383 + 0.0014731
[600]	cv_agg's multi_logloss: 0.173911 + 0.000978608	cv_agg's f1_score: 0.810874 + 0.00140886
[650]	cv_agg's multi_logloss: 0.172995 + 0.000911217	cv_agg's f1_score: 0.812316 + 0.00103285
[700]	cv_agg's multi_logloss: 0.17221 + 0.00085273	cv_agg's f1_score: 0.813637 + 0.000811492
[750]	cv_agg's multi_logloss: 0.171527 + 0.000851802	cv_agg's f1_score: 0.814692 + 0.00100554
[800]	cv_agg's multi_logloss: 0.171044 + 0.000895539	cv_agg's f1_score: 0.81556 + 0.00120356
[850]	cv_agg's multi_logloss: 0.170673 + 0.000908041	cv_agg's f1_score: 0.81644 + 0.00101881
[900]	cv_agg's multi_logloss: 0.170342 + 0.000918312	cv_agg's f1_score: 0.816903 + 0.000991058
[950]	cv_agg's multi_logloss: 0.170075 + 0.000919927	cv_agg's f1_score: 0.81756 + 0.00113742
[1000]	cv_agg's multi_logloss: 0.16985 + 0.000970451	cv_agg's f1_s

[100]	cv_agg's multi_logloss: 0.216297 + 0.000597077	cv_agg's f1_score: 0.770654 + 0.00139784
[150]	cv_agg's multi_logloss: 0.200801 + 0.000886027	cv_agg's f1_score: 0.782817 + 0.00151839
[200]	cv_agg's multi_logloss: 0.192797 + 0.000774279	cv_agg's f1_score: 0.789363 + 0.000967992
[250]	cv_agg's multi_logloss: 0.187871 + 0.000908991	cv_agg's f1_score: 0.794459 + 0.0010049
[300]	cv_agg's multi_logloss: 0.184453 + 0.000883205	cv_agg's f1_score: 0.797778 + 0.000766388
[350]	cv_agg's multi_logloss: 0.181826 + 0.000894976	cv_agg's f1_score: 0.80063 + 0.000983663
[400]	cv_agg's multi_logloss: 0.179757 + 0.000901988	cv_agg's f1_score: 0.803643 + 0.00107282
[450]	cv_agg's multi_logloss: 0.177984 + 0.000882437	cv_agg's f1_score: 0.805638 + 0.000855484
[500]	cv_agg's multi_logloss: 0.176555 + 0.000847463	cv_agg's f1_score: 0.807883 + 0.000866447
[550]	cv_agg's multi_logloss: 0.17532 + 0.000859762	cv_agg's f1_score: 0.809328 + 0.000685973
[600]	cv_agg's multi_logloss: 0.174365 + 0.000837976	cv_a

[800]	cv_agg's multi_logloss: 0.168969 + 0.00133404	cv_agg's f1_score: 0.820052 + 0.00115475
[850]	cv_agg's multi_logloss: 0.168835 + 0.00134701	cv_agg's f1_score: 0.820427 + 0.0011645
[900]	cv_agg's multi_logloss: 0.168807 + 0.00136353	cv_agg's f1_score: 0.820961 + 0.00123481
[950]	cv_agg's multi_logloss: 0.168882 + 0.00145273	cv_agg's f1_score: 0.821474 + 0.00116406
[50]	cv_agg's multi_logloss: 0.259952 + 0.000672159	cv_agg's f1_score: 0.752437 + 0.00182334
[100]	cv_agg's multi_logloss: 0.210557 + 0.000599211	cv_agg's f1_score: 0.776183 + 0.00100361
[150]	cv_agg's multi_logloss: 0.195065 + 0.00113901	cv_agg's f1_score: 0.789261 + 0.00121184
[200]	cv_agg's multi_logloss: 0.186817 + 0.000914105	cv_agg's f1_score: 0.796557 + 0.00093828
[250]	cv_agg's multi_logloss: 0.181739 + 0.000958921	cv_agg's f1_score: 0.801332 + 0.00112756
[300]	cv_agg's multi_logloss: 0.178349 + 0.00104159	cv_agg's f1_score: 0.804999 + 0.000730847
[350]	cv_agg's multi_logloss: 0.175905 + 0.00112754	cv_agg's f1_sco

[400]	cv_agg's multi_logloss: 0.17631 + 0.00108772	cv_agg's f1_score: 0.80733 + 0.00170088
[450]	cv_agg's multi_logloss: 0.174802 + 0.00105888	cv_agg's f1_score: 0.8095 + 0.00120525
[500]	cv_agg's multi_logloss: 0.173586 + 0.00108698	cv_agg's f1_score: 0.811742 + 0.00103953
[550]	cv_agg's multi_logloss: 0.172578 + 0.0011146	cv_agg's f1_score: 0.812955 + 0.0010584
[600]	cv_agg's multi_logloss: 0.171666 + 0.00121877	cv_agg's f1_score: 0.814099 + 0.00135867
[650]	cv_agg's multi_logloss: 0.170917 + 0.00128446	cv_agg's f1_score: 0.815355 + 0.00126914
[700]	cv_agg's multi_logloss: 0.170308 + 0.0013309	cv_agg's f1_score: 0.816466 + 0.00138228
[750]	cv_agg's multi_logloss: 0.169848 + 0.00139681	cv_agg's f1_score: 0.81729 + 0.00165153
[800]	cv_agg's multi_logloss: 0.169439 + 0.00143157	cv_agg's f1_score: 0.81806 + 0.00143868
[850]	cv_agg's multi_logloss: 0.16914 + 0.00143714	cv_agg's f1_score: 0.818557 + 0.00159863
[900]	cv_agg's multi_logloss: 0.168835 + 0.00147005	cv_agg's f1_score: 0.819407 

[350]	cv_agg's multi_logloss: 0.172045 + 0.00138025	cv_agg's f1_score: 0.814176 + 0.000795476
[400]	cv_agg's multi_logloss: 0.170633 + 0.00139589	cv_agg's f1_score: 0.81586 + 0.000735494
[450]	cv_agg's multi_logloss: 0.169587 + 0.00139012	cv_agg's f1_score: 0.817582 + 0.0010831
[500]	cv_agg's multi_logloss: 0.168874 + 0.00152238	cv_agg's f1_score: 0.819193 + 0.00119143
[550]	cv_agg's multi_logloss: 0.168359 + 0.00163203	cv_agg's f1_score: 0.820443 + 0.0011969
[600]	cv_agg's multi_logloss: 0.168123 + 0.00164007	cv_agg's f1_score: 0.821523 + 0.00106066
[650]	cv_agg's multi_logloss: 0.168047 + 0.00165148	cv_agg's f1_score: 0.822159 + 0.000876638


In [190]:
submit_lgb = []
for line in cv_pred:
    submit_lgb.append(np.argmax(np.bincount(line)))

In [191]:
submit_lgb = le.inverse_transform(submit_lgb)
df_test = test_id[['user_id']]
df_test['predict_service'] = submit_lgb

In [192]:
df_test.to_csv('lgb_vote.csv', index=None)

## XGB程序

In [173]:
import xgboost as xgb

def XGB():
    clf = xgb.XGBClassifier(max_depth=12, learning_rate=0.1,
                            n_estimators=500, silent=True,
                            objective="multi:softmax",
                            nthread=4, gamma=0,
                            max_delta_step=0, subsample=1, colsample_bytree=0.9, colsample_bylevel=0.9,
                            reg_alpha=1, reg_lambda=1, scale_pos_weight=1,
                            base_score=0.5, seed=2018, missing=None,num_class=11)
    return clf

fee_min 0.7422

In [174]:
cv_pred = []
xgb_oof_train = np.zeros((len(train_x),))
for index, (train_index, valid_index) in enumerate(skf.split(train_x, train_y_le)):
    print(index)
    x_train, x_valid, y_train, y_valid = train_x.loc[train_index], train_x.loc[valid_index], train_y_le[train_index], train_y_le[valid_index]
    clf = XGB()
    clf.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], verbose=1)
    x_pred = clf.predict(x_valid)
#     x_pred = [list(x).index(max(x)) for x in x_pred]
#     x_score.append(clf.best_score['valid_0']['multi_logloss'])
    y_test = clf.predict(test_x)
    xgb_oof_train[valid_index] = x_pred
    
    if index == 0:
        cv_pred = np.array(y_test).reshape(-1, 1)
    else:
        cv_pred = np.hstack((cv_pred, np.array(y_test).reshape(-1, 1)))

0


XGBoostError: b'[10:07:29] src/metric/metric.cc:21: Unknown metric function <function f1_score_vali at 0x000001BC93185C80>'

In [131]:
submit_xgb = []
for line in cv_pred:
    submit_xgb.append(np.argmax(np.bincount(line)))

In [140]:
xgb_oof_train.astype(int)

array([6, 3, 4, ..., 3, 0, 8])

In [152]:
trpre_xgb = le.inverse_transform(xgb_oof_train.astype(int))
df_train = train_id[['user_id']]
df_train['predict_service'] = trpre_xgb

In [146]:
tr_label = train[['user_id', 'current_service']]

In [147]:
submit_xgb = le.inverse_transform(submit_xgb)
df_test = test_id[['user_id']]
df_test['predict_service'] = submit_xgb

In [169]:
df_test.to_csv('xgb_vote_0.743.csv', index=None)

## 存储临时数据

In [166]:
submit_xgb = np.array(submit_xgb)
np.save('submit_xgb_0.743.pny', submit_xgb)

In [168]:
np.save('xgb_oof_train_0.743.pny', xgb_oof_train)
# np.save('lgb_oof_train.pny', lgb_oof_train)

In [40]:
submit_lgb = np.array(submit_lgb)
np.save('submit_lgb.pny', submit_lgb)

In [155]:
score(tr_label, df_train)

0.9592992045288177 99999825
0.9976285410826529 90063345
0.9964324125809921 90109916
0.944666371315714 89950166
0.9676266076865978 89950168
0.9079679569497952 89950167
0.9715852442671985 90155946
0.8714209766841345 99999828
0.86108567909044 99999826
0.8289593598912556 99999827
0.8079105846199598 99999830


0.8454941158991052

## 第二层XGB预测

In [36]:
multi_train = np.concatenate((lgb_oof_train.reshape(-1, 1), xgb_oof_train.reshape(-1, 1)), axis=1)
multi_test = np.concatenate((np.array(submit_lgb).reshape(-1, 1), np.array(submit_xgb).reshape(-1, 1)), axis=1)

In [37]:
def XGB2():
    clf = xgb.XGBClassifier(max_depth=10, learning_rate=0.1,
                            n_estimators=300, silent=True,
                            objective="multi:softmax",
                            nthread=4, gamma=0,
                            max_delta_step=0, subsample=1, colsample_bytree=0.9, colsample_bylevel=0.9,
                            reg_alpha=1, reg_lambda=1, scale_pos_weight=1,
                            base_score=0.5, seed=2018, missing=None,num_class=11)
    return clf

In [44]:
clf = XGB2()
clf.fit(multi_train, train_y_le)
submit = clf.predict(multi_test)

In [45]:
submit_multi = le.inverse_transform(submit)
df_test = test_id[['user_id']]
df_test['predict_service'] = submit_multi

In [ ]:
df_test.to_csv('multi_vote.csv', index=None)

后面的都不用管

## GridSearchCV选择参数

In [80]:
train_tra_x = train_x[1:int(4/5*len(train_x))]
train_val_x = train_x[int(4/5*len(train_x)):]

In [81]:
train_tra_y = train_y_le[1:int(4/5*len(train_x))]
train_val_y = train_y_le[int(4/5*len(train_x)):]

In [73]:
from sklearn.model_selection import GridSearchCV

model_lgb = lgb.LGBMClassifier(objective='multiclass', learning_rate=0.05, n_estimators=2000, reg_alpha=0.1, reg_lambda=0.2)
param = {'max_depth':range(7, 9), 'num_leaves':range(100, 130, 20)}
search = GridSearchCV(estimator=model_lgb, param_grid=param, cv=5, verbose=1, n_jobs=2)

In [74]:
search.fit(train4_x, train4_y_le)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed: 619.7min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.05, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=2000, n_jobs=-1, num_leaves=31,
        objective='multiclass', random_state=None, reg_alpha=0.1,
        reg_lambda=0.2, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=0),
       fit_params=None, iid=True, n_jobs=2,
       param_grid={'max_depth': range(7, 9), 'num_leaves': range(100, 130, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [75]:
search.best_params_

{'max_depth': 8, 'num_leaves': 120}

## cv实验参数的合理性

In [20]:
column_str = ['service_type', 'is_mix_service', 'many_over_bill', 'contract_type', 'is_promise_low_consume', 'net_service', 'gender']
train_data = lgb.Dataset(train_tra_x, label=train_tra_y, feature_name=list(train_tra_x.columns), categorical_feature=column_str)
validate_data = lgb.Dataset(train_val_x, label=train_val_y, feature_name=list(train_tra_x.columns), categorical_feature=column_str)
train_all = lgb.Dataset(train_x, train_y_le, feature_name=list(train_tra_x.columns), categorical_feature=column_str)

In [21]:
params4 = {'learning_rate':0.1, 
          'lambda_l1':0.18,
          'lambda_l2':0.23,
          'max_depth':7,
          'num_leaves':100, 
           'n_estimators':3000, 
          'objective':'multiclass',
          'num_class':11
          }

In [22]:
cv_result = lgb.cv(params4, train_all4, nfold=5, verbose_eval=50, early_stopping_rounds=50)

KeyboardInterrupt: 

In [145]:
# print('best n_estinators:', len(cv_result['multi_logloss-mean']))

best n_estinators: 700


0.16202

In [34]:
lgb4 = lgb.train(params4, train_data4, valid_sets=[validate_data4])
# lgb4 = lgb.train(params4, train_all4)

[1]	valid_0's multi_logloss: 2.18645
[2]	valid_0's multi_logloss: 2.02188
[3]	valid_0's multi_logloss: 1.88648
[4]	valid_0's multi_logloss: 1.7709
[5]	valid_0's multi_logloss: 1.67019
[6]	valid_0's multi_logloss: 1.58073
[7]	valid_0's multi_logloss: 1.50067
[8]	valid_0's multi_logloss: 1.42875
[9]	valid_0's multi_logloss: 1.36316
[10]	valid_0's multi_logloss: 1.30311
[11]	valid_0's multi_logloss: 1.24737
[12]	valid_0's multi_logloss: 1.19622
[13]	valid_0's multi_logloss: 1.14849
[14]	valid_0's multi_logloss: 1.10405
[15]	valid_0's multi_logloss: 1.06263
[16]	valid_0's multi_logloss: 1.02373
[17]	valid_0's multi_logloss: 0.986771
[18]	valid_0's multi_logloss: 0.952232
[19]	valid_0's multi_logloss: 0.91976
[20]	valid_0's multi_logloss: 0.889169
[21]	valid_0's multi_logloss: 0.860363
[22]	valid_0's multi_logloss: 0.83319
[23]	valid_0's multi_logloss: 0.807429
[24]	valid_0's multi_logloss: 0.783102
[25]	valid_0's multi_logloss: 0.760027
[26]	valid_0's multi_logloss: 0.738209
[27]	valid_0's

[210]	valid_0's multi_logloss: 0.211576
[211]	valid_0's multi_logloss: 0.211335
[212]	valid_0's multi_logloss: 0.211087
[213]	valid_0's multi_logloss: 0.210867
[214]	valid_0's multi_logloss: 0.210612
[215]	valid_0's multi_logloss: 0.210353
[216]	valid_0's multi_logloss: 0.210114
[217]	valid_0's multi_logloss: 0.209949
[218]	valid_0's multi_logloss: 0.209694
[219]	valid_0's multi_logloss: 0.209505
[220]	valid_0's multi_logloss: 0.209324
[221]	valid_0's multi_logloss: 0.209059
[222]	valid_0's multi_logloss: 0.20885
[223]	valid_0's multi_logloss: 0.208659
[224]	valid_0's multi_logloss: 0.208473
[225]	valid_0's multi_logloss: 0.20824
[226]	valid_0's multi_logloss: 0.208052
[227]	valid_0's multi_logloss: 0.207787
[228]	valid_0's multi_logloss: 0.207548
[229]	valid_0's multi_logloss: 0.207347
[230]	valid_0's multi_logloss: 0.207161
[231]	valid_0's multi_logloss: 0.207004
[232]	valid_0's multi_logloss: 0.206784
[233]	valid_0's multi_logloss: 0.206524
[234]	valid_0's multi_logloss: 0.206273
[2

[416]	valid_0's multi_logloss: 0.185364
[417]	valid_0's multi_logloss: 0.185281
[418]	valid_0's multi_logloss: 0.185223
[419]	valid_0's multi_logloss: 0.185138
[420]	valid_0's multi_logloss: 0.185085
[421]	valid_0's multi_logloss: 0.185015
[422]	valid_0's multi_logloss: 0.184932
[423]	valid_0's multi_logloss: 0.184868
[424]	valid_0's multi_logloss: 0.184808
[425]	valid_0's multi_logloss: 0.184746
[426]	valid_0's multi_logloss: 0.184698
[427]	valid_0's multi_logloss: 0.184642
[428]	valid_0's multi_logloss: 0.184584
[429]	valid_0's multi_logloss: 0.184503
[430]	valid_0's multi_logloss: 0.184439
[431]	valid_0's multi_logloss: 0.184386
[432]	valid_0's multi_logloss: 0.18433
[433]	valid_0's multi_logloss: 0.18428
[434]	valid_0's multi_logloss: 0.184207
[435]	valid_0's multi_logloss: 0.184149
[436]	valid_0's multi_logloss: 0.184078
[437]	valid_0's multi_logloss: 0.184016
[438]	valid_0's multi_logloss: 0.183966
[439]	valid_0's multi_logloss: 0.18391
[440]	valid_0's multi_logloss: 0.183853
[44

[622]	valid_0's multi_logloss: 0.175234
[623]	valid_0's multi_logloss: 0.175206
[624]	valid_0's multi_logloss: 0.175162
[625]	valid_0's multi_logloss: 0.175134
[626]	valid_0's multi_logloss: 0.175092
[627]	valid_0's multi_logloss: 0.175048
[628]	valid_0's multi_logloss: 0.175008
[629]	valid_0's multi_logloss: 0.174966
[630]	valid_0's multi_logloss: 0.174936
[631]	valid_0's multi_logloss: 0.174907
[632]	valid_0's multi_logloss: 0.174878
[633]	valid_0's multi_logloss: 0.174863
[634]	valid_0's multi_logloss: 0.174831
[635]	valid_0's multi_logloss: 0.174797
[636]	valid_0's multi_logloss: 0.174754
[637]	valid_0's multi_logloss: 0.174713
[638]	valid_0's multi_logloss: 0.174681
[639]	valid_0's multi_logloss: 0.174646
[640]	valid_0's multi_logloss: 0.174614
[641]	valid_0's multi_logloss: 0.174579
[642]	valid_0's multi_logloss: 0.174541
[643]	valid_0's multi_logloss: 0.174522
[644]	valid_0's multi_logloss: 0.17449
[645]	valid_0's multi_logloss: 0.174455
[646]	valid_0's multi_logloss: 0.174429
[

[828]	valid_0's multi_logloss: 0.169318
[829]	valid_0's multi_logloss: 0.16929
[830]	valid_0's multi_logloss: 0.169276
[831]	valid_0's multi_logloss: 0.169266
[832]	valid_0's multi_logloss: 0.169218
[833]	valid_0's multi_logloss: 0.169202
[834]	valid_0's multi_logloss: 0.169182
[835]	valid_0's multi_logloss: 0.169172
[836]	valid_0's multi_logloss: 0.169158
[837]	valid_0's multi_logloss: 0.169136
[838]	valid_0's multi_logloss: 0.169109
[839]	valid_0's multi_logloss: 0.169084
[840]	valid_0's multi_logloss: 0.169056
[841]	valid_0's multi_logloss: 0.169038
[842]	valid_0's multi_logloss: 0.169017
[843]	valid_0's multi_logloss: 0.168999
[844]	valid_0's multi_logloss: 0.168975
[845]	valid_0's multi_logloss: 0.168961
[846]	valid_0's multi_logloss: 0.168937
[847]	valid_0's multi_logloss: 0.168909
[848]	valid_0's multi_logloss: 0.16889
[849]	valid_0's multi_logloss: 0.168857
[850]	valid_0's multi_logloss: 0.168836
[851]	valid_0's multi_logloss: 0.168814
[852]	valid_0's multi_logloss: 0.168788
[8

...

[1234]	valid_0's multi_logloss: 0.162319
[1235]	valid_0's multi_logloss: 0.162307
[1236]	valid_0's multi_logloss: 0.162293
[1237]	valid_0's multi_logloss: 0.162284
[1238]	valid_0's multi_logloss: 0.162267
[1239]	valid_0's multi_logloss: 0.162255
[1240]	valid_0's multi_logloss: 0.162241
[1241]	valid_0's multi_logloss: 0.162235
[1242]	valid_0's multi_logloss: 0.162223
[1243]	valid_0's multi_logloss: 0.162207
[1244]	valid_0's multi_logloss: 0.162192
[1245]	valid_0's multi_logloss: 0.162181
[1246]	valid_0's multi_logloss: 0.162169
[1247]	valid_0's multi_logloss: 0.162142
[1248]	valid_0's multi_logloss: 0.162126
[1249]	valid_0's multi_logloss: 0.162103
[1250]	valid_0's multi_logloss: 0.162087
[1251]	valid_0's multi_logloss: 0.162074
[1252]	valid_0's multi_logloss: 0.162049
[1253]	valid_0's multi_logloss: 0.162035
[1254]	valid_0's multi_logloss: 0.162022
[1255]	valid_0's multi_logloss: 0.162014
[1256]	valid_0's multi_logloss: 0.162005
[1257]	valid_0's multi_logloss: 0.16199
[1258]	valid_0's 

[1435]	valid_0's multi_logloss: 0.160341
[1436]	valid_0's multi_logloss: 0.160337
[1437]	valid_0's multi_logloss: 0.160329
[1438]	valid_0's multi_logloss: 0.160318
[1439]	valid_0's multi_logloss: 0.160308
[1440]	valid_0's multi_logloss: 0.160302
[1441]	valid_0's multi_logloss: 0.160294
[1442]	valid_0's multi_logloss: 0.160289
[1443]	valid_0's multi_logloss: 0.160285
[1444]	valid_0's multi_logloss: 0.160277
[1445]	valid_0's multi_logloss: 0.160264
[1446]	valid_0's multi_logloss: 0.160252
[1447]	valid_0's multi_logloss: 0.160248
[1448]	valid_0's multi_logloss: 0.160238
[1449]	valid_0's multi_logloss: 0.160237
[1450]	valid_0's multi_logloss: 0.160225
[1451]	valid_0's multi_logloss: 0.160213
[1452]	valid_0's multi_logloss: 0.160201
[1453]	valid_0's multi_logloss: 0.160191
[1454]	valid_0's multi_logloss: 0.160184
[1455]	valid_0's multi_logloss: 0.160177
[1456]	valid_0's multi_logloss: 0.16017
[1457]	valid_0's multi_logloss: 0.16017
[1458]	valid_0's multi_logloss: 0.160163
[1459]	valid_0's m

[1636]	valid_0's multi_logloss: 0.159272
[1637]	valid_0's multi_logloss: 0.159269
[1638]	valid_0's multi_logloss: 0.159267
[1639]	valid_0's multi_logloss: 0.159265
[1640]	valid_0's multi_logloss: 0.159264
[1641]	valid_0's multi_logloss: 0.159265
[1642]	valid_0's multi_logloss: 0.159264
[1643]	valid_0's multi_logloss: 0.159262
[1644]	valid_0's multi_logloss: 0.15926
[1645]	valid_0's multi_logloss: 0.15926
[1646]	valid_0's multi_logloss: 0.159257
[1647]	valid_0's multi_logloss: 0.159256
[1648]	valid_0's multi_logloss: 0.159252
[1649]	valid_0's multi_logloss: 0.159254
[1650]	valid_0's multi_logloss: 0.159253
[1651]	valid_0's multi_logloss: 0.159255
[1652]	valid_0's multi_logloss: 0.159255
[1653]	valid_0's multi_logloss: 0.159253
[1654]	valid_0's multi_logloss: 0.159249
[1655]	valid_0's multi_logloss: 0.159248
[1656]	valid_0's multi_logloss: 0.159248
[1657]	valid_0's multi_logloss: 0.159249
[1658]	valid_0's multi_logloss: 0.159246
[1659]	valid_0's multi_logloss: 0.159245
[1660]	valid_0's m

[1837]	valid_0's multi_logloss: 0.159121
[1838]	valid_0's multi_logloss: 0.159121
[1839]	valid_0's multi_logloss: 0.159121
[1840]	valid_0's multi_logloss: 0.159121
[1841]	valid_0's multi_logloss: 0.159121
[1842]	valid_0's multi_logloss: 0.159121
[1843]	valid_0's multi_logloss: 0.159121
[1844]	valid_0's multi_logloss: 0.159121
[1845]	valid_0's multi_logloss: 0.159121
[1846]	valid_0's multi_logloss: 0.159121
[1847]	valid_0's multi_logloss: 0.159121
[1848]	valid_0's multi_logloss: 0.159121
[1849]	valid_0's multi_logloss: 0.159121
[1850]	valid_0's multi_logloss: 0.159121
[1851]	valid_0's multi_logloss: 0.159121
[1852]	valid_0's multi_logloss: 0.159121
[1853]	valid_0's multi_logloss: 0.159121
[1854]	valid_0's multi_logloss: 0.159121
[1855]	valid_0's multi_logloss: 0.159121
[1856]	valid_0's multi_logloss: 0.159121
[1857]	valid_0's multi_logloss: 0.159121
[1858]	valid_0's multi_logloss: 0.159121
[1859]	valid_0's multi_logloss: 0.159121
[1860]	valid_0's multi_logloss: 0.159121
[1861]	valid_0's

KeyboardInterrupt: 

In [49]:
val_lgb4 = lgb4.predict(train4_val_x)
val_lgb4 = [list(x).index(max(x)) for x in val_lgb4]

In [50]:
validation4 = train4_id[int(4/5*len(train4_x_stand)):]
validation4['predict_service'] = le4.inverse_transform(val_lgb4)

In [51]:
val_y4 = train4_id[int(4/5*len(train4_x_stand)):]
val_y4['current_service'] = le4.inverse_transform(train4_val_y)

use_situation 上升

pay caller_time 都没什么用 age_like会降分

total_fee: 0.84129756 0.7347

online_mix:0.83459 0.7340

baseline_7_100: 0.829 线上0.738

In [53]:
import matplotlib.pyplot as plt

print(pd.DataFrame({
        'column': train4_feature,
        'importance': lgb4.feature_importance(),
    }).sort_values(by='importance'))

# plt.plot(sorted(lgb4.feature_importance()))

                    column  importance
23    former_complaint_num          92
12             net_service         169
13               pay_times         398
11  is_promise_low_consume         467
22         complaint_level         494
1           is_mix_service         572
24    former_complaint_fee         658
20                  gender         851
0             service_type         963
17       local_caller_time        1276
21                     age        1812
8           many_over_bill        2016
14                 pay_num        2050
16    local_trafffic_month        3053
10           contract_time        3585
2              online_time        3937
18    service1_caller_time        4068
6              4_total_fee        4334
7            month_traffic        5076
4              2_total_fee        5257
9            contract_type        5457
19    service2_caller_time        6945
5              3_total_fee        7003
15      last_month_traffic        7700
3              1_total_fe

# 测试集

In [39]:
test4 = pd.read_csv('test4_featured.csv')
# test1 = pd.read_csv('test1_featured.csv')

In [41]:
test4_id = test4[['user_id']]
test4_x = test4.drop(['user_id'], axis=1)

In [42]:
test4_x_stand = test4_x

In [43]:
pre_test4 = lgb4.predict(test4_x_stand)

In [44]:
pre_test4 = [list(x).index(max(x)) for x in pre_test4]

In [45]:
pre_test4 = le4.inverse_transform(pre_test4)

In [46]:
predict4 = test4_id
predict4['predict_service'] = pre_test4

In [47]:
predict4.to_csv('lgb_all1010_8_150.csv', index=None)

In [154]:
def score(tr, te):
    labels = tr['current_service'].unique()
    f1_score = 0
    for label in labels:
        tp = 0
        fp = 0
        fn = 0
        sub_tr = tr[tr['current_service'] == label]['current_service']
        index_tr = tr[tr['current_service'] == label].index.tolist()
        sub_te = te['predict_service'][index_tr]
        for la in sub_te.values:
            if la == label:
                tp = tp + 1
            else:
                fn = fn + 1
        index_te = te[te['predict_service'] == label].index.tolist()
        fp_list = set(index_te).difference(index_tr)
        fp = len(fp_list)
        precision = tp/(tp+fp)
        recall = tp/(tp+fn)
        f1_score = f1_score + (2*precision*recall)/(precision+recall)
        print((2*precision*recall)/(precision+recall), label)
    final_score = (1/(len(labels))*f1_score)**2
    
    return final_score

In [119]:
from sklearn.metrics import f1_score
import numpy as np

# 自定义F1评价函数
def f1_score_vali(preds, data_vali):
    labels = data_vali['current_service'].unique()
    preds = np.argmax(preds.reshape(11, -1), axis=0)
    score_vali = f1_score(y_true=labels, y_pred=preds, average='weighted')
    return 'f1_score', score_vali, True

In [149]:
def score(tr, te):
    labels = tr['current_service'].unique()
    f1_score = 0
    for label in labels:
        tp = 0
        fp = 0
        fn = 0
        sub_tr = tr[tr['current_service'] == label]['current_service']
        index_tr = tr[tr['current_service'] == label].index.tolist()
        sub_te = te['predict_service'][index_tr]
        for la in sub_te.values:
            if la == label:
                tp = tp + 1
            else:
                fn = fn + 1
        index_te = te[te['predict_service'] == label].index.tolist()
        fp_list = set(index_te).difference(index_tr)
        fp = len(fp_list)
        precision = tp/(tp+fp)
        recall = tp/(tp+fn)
        f1_score = f1_score + (2*precision*recall)/(precision+recall)
    final_score = (1/(len(labels))*f1_score)**2
    
    return final_score